In [1]:
# Importing libraries
import xml.etree.ElementTree as ET
from openpyxl import Workbook
from datetime import datetime

In [2]:
import xml.etree.ElementTree as ET
import openpyxl

class XMLReader:
    def __init__(self, file_path):
        self.file_path = file_path
        self.root = None
        self.workbook = None
        self.sheet = None
        self.current_row = 2
    
    def parse_xml(self):
        try:
            tree = ET.parse(self.file_path)
            self.root = tree.getroot()
            print("XML file parsed successfully.")
        except ET.ParseError as e:
            print(f"Error parsing XML file: {e}")
    
    def create_excel_file(self):
        self.workbook = openpyxl.Workbook()
        self.sheet = self.workbook.active
        self.create_header_for_excel()
        
    def create_header_for_excel(self):
        self.sheet.cell(1, 1).value = "Date"
        self.sheet.cell(1, 2).value = "Transaction Type"
        self.sheet.cell(1, 3).value = "Vch No."
        self.sheet.cell(1, 4).value = "Ref No."
        self.sheet.cell(1, 5).value = "Ref Type"
        self.sheet.cell(1, 6).value = "Ref Date"
        self.sheet.cell(1, 7).value = "Debtor"
        self.sheet.cell(1, 8).value = "Ref Amount"
        self.sheet.cell(1, 9).value = "Amount"
        self.sheet.cell(1, 10).value = "Particulars"
        self.sheet.cell(1, 11).value = "Vch Type"
        self.sheet.cell(1, 12).value = "Amount Verified"
        
    def write_elements_to_row(self, elements):
        self.sheet.cell(row=self.current_row, column=1).value = datetime.strptime(elements[0], "%Y%m%d").strftime("%d-%m-%Y")
        self.sheet.cell(row=self.current_row, column=2).value = elements[1]
        self.sheet.cell(row=self.current_row, column=3).value = elements[2]
        self.sheet.cell(row=self.current_row, column=4).value = elements[3]
        self.sheet.cell(row=self.current_row, column=5).value = elements[4]
        self.sheet.cell(row=self.current_row, column=6).value = elements[5]
        self.sheet.cell(row=self.current_row, column=7).value = elements[6]
        self.sheet.cell(row=self.current_row, column=8).value = elements[7]
        self.sheet.cell(row=self.current_row, column=9).value = elements[8]
        self.sheet.cell(row=self.current_row, column=10).value =elements[9]
        self.sheet.cell(row=self.current_row, column=11).value =elements[10]
        self.sheet.cell(row=self.current_row, column=12).value =elements[11]
        
    def write_data_to_excel(self, data):
        for row_data in data:
            self.write_elements_to_row(tuple(row_data))
            self.current_row += 1
            
    def iterate_elements(self):
        if self.root is None:
            print("XML file has not been parsed yet.")
            return
        
        # find BODY node
        body_node = self.root.find("BODY")
        if body_node is None:
            print("BODY node not found in XML.")
            return
        
        # find IMPORTDATA node
        import_data_node = body_node.find("IMPORTDATA")
        if import_data_node is None:
            print("IMPORTDATA node not found in XML.")
            return
        
        # find REQUESTDATA node
        request_data_node = import_data_node.find("REQUESTDATA")
        if request_data_node is None:
            print("REQUESTDATA node not found in XML.")
            return
        
        for node in request_data_node.iter():
            # iterate until REQUESTDATA node is found
            if node.tag != "TALLYMESSAGE":
                continue
              
            # find VOUCHER node
            vocher_node = node.find("VOUCHER")
            if vocher_node is None:
                print("VOUCHER node not found in XML.")
                continue
                
            # find VOUCHERTYPENAME element
            vocher_type_element = vocher_node.find("VOUCHERTYPENAME")
            if vocher_type_element is None:
                print("VOUCHERTYPENAME node not found in XML.")
                continue
            elif vocher_type_element.text != "Receipt":
                continue
            
            # find DATE element
            date_element = vocher_node.find("DATE")
            if date_element is None: 
                continue
                
            # find VOUCHERNUMBER element
            voucher_no_element = vocher_node.find("VOUCHERNUMBER")
            if voucher_no_element is None: 
                continue
                
            # find PARTYLEDGERNAME element
            party_ledger_name_element = vocher_node.find("PARTYLEDGERNAME")
            if party_ledger_name_element is None: 
                continue
                
            # find ALLLEDGERENTRIES.LIST node
            all_ledger_enteries_node = vocher_node.findall("ALLLEDGERENTRIES.LIST")
            if all_ledger_enteries_node is None: 
                continue               
            
            parent_amt_element = 0 
            child_amt_total = 0
            no_of_child_elements = 0
            
            list_data_to_write = []
            
            # iterate through all the ALLLEDGERENTRIES.LIST
            for ledger_entry_node in all_ledger_enteries_node:
                # find LEDGERNAME element
                ledger_name_element = ledger_entry_node.find("LEDGERNAME")
                
                # find BILLALLOCATIONS.LIST node
                bill_allocation_enteries_node = ledger_entry_node.findall("BILLALLOCATIONS.LIST")
                
                if isinstance(bill_allocation_enteries_node, list):
                                        
                    # iterate through all the BILLALLOCATIONS.LIST
                    for bill_allocation_entry_node in bill_allocation_enteries_node:
                        
                        if bill_allocation_entry_node is None:
                            continue
                        
                        # find NAME elements
                        ref_no_element = bill_allocation_entry_node.find("NAME")
                        if ref_no_element is None:
                            
                            # find required elements
                            other_amt_element = ledger_entry_node.find("AMOUNT")
                            other_row_elements = (date_element.text, "Other", voucher_no_element.text, "NA", "NA", "NA", ledger_name_element.text, "NA", other_amt_element.text, ledger_name_element.text, vocher_type_element.text, "NA")
                        
                            no_of_child_elements += 1
                            list_data_to_write.append(other_row_elements)
                            
                        else:
                            # find required elements
                            parent_amt_element = ledger_entry_node.find("AMOUNT")
                            ref_type_element = bill_allocation_entry_node.find("BILLTYPE")
                            ref_amt_element = bill_allocation_entry_node.find("AMOUNT")
                            
                            child_row_elements = (date_element.text, "Child", voucher_no_element.text, ref_no_element.text, ref_type_element.text, "", ledger_name_element.text, ref_amt_element.text, "NA", ledger_name_element.text, vocher_type_element.text, "NA")
                            list_data_to_write.append(child_row_elements)
                            
                            no_of_child_elements += 1
                            child_amt_total += float(ref_amt_element.text)                           
                            
                    
            is_verified = "NA"
            if child_amt_total == float(parent_amt_element.text):
                is_verified = "Yes"
                
            parent_row_elements = (date_element.text, "Parent", voucher_no_element.text, "NA", "NA", "NA", ledger_name_element.text, "NA", parent_amt_element.text, ledger_name_element.text, vocher_type_element.text, is_verified)
            list_data_to_write.insert(0, parent_row_elements)
            
            #write data to excel
            self.write_data_to_excel(list_data_to_write)
    
    def save_excel_file(self, file_path):
        if self.workbook is not None:
            self.workbook.save(file_path)
            print(f"Excel file saved successfully: {file_path}")
        else:
            print("No Excel file to save.")


In [3]:
xml_file_path = "Input.xml"
excel_file_path = "Output.xlsx"

reader = XMLReader(xml_file_path)
reader.parse_xml()
reader.create_excel_file()
reader.iterate_elements()
reader.save_excel_file(excel_file_path)

XML file parsed successfully.
Excel file saved successfully: Output.xlsx
